In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from FINDER import FINDER
from FINDER_test_utils import *
import os
import tqdm
import numpy as np
import pandas as pd
import tsplib95
import networkx as nx
import re


/root/miniconda3/envs/findervenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/envs/findervenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/envs/findervenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/envs/findervenv/lib/python3.7/site-

In [3]:
model_path =  'best_models/tsp_2d/nrange_15_20_len_1038117/'
model_file, model_base_path, tour_length = get_model_file(model_path)
config_path = model_path + 'config.txt'
dqn = FINDER(config_path=config_path)
# load model into Finder
print(model_file)
best_model = dqn.LoadModel(model_path=model_path+model_file)

Best model file: nrange_15_20_iter_19500_len_1038117.ckpt
{'help_func': 1, 'net_type': 'FINDER', 'max_bp_iter': 3, 'aggregatorID': 0, 'node_init_dim': 6, 'edge_init_dim': 4, 'state_init_dim': 1, 'node_embed_dim': 64, 'edge_embed_dim': 64, 'embeddingMethod': 3, 'ignore_covered_edges': 1, 'selected_nodes_inclusion': 2, 'IsHuberloss': 0, 'BATCH_SIZE': 64, 'initialization_stddev': 0.01, 'MAX_ITERATION': 20000, 'LEARNING_RATE': 0.0001, 'Alpha': 0.001, 'save_interval': 300, 'num_env': 1, 'g_type': 'tsp_2d', 'NUM_MIN': 15, 'NUM_MAX': 20, 'n_generator': 1000, 'decoder': 0, 'REG_HIDDEN': 32, 'IsDoubleDQN': 0, 'N_STEP': 1, 'GAMMA': 0.1, 'UPDATE_TIME': 1000, 'eps_start': 1.0, 'eps_end': 1.0, 'eps_step': 10000.0, 'MEMORY_SIZE': 150000, 'reward_normalization': 'max', 'reward_sign': 1, 'valid_path': 'valid_sets/synthetic_nrange_40_50_100/', 'valid_scale_fac': 1e-06, 'n_valid': 100, 'IsPrioritizedSampling': 0, 'epsilon': 1e-07, 'alpha': 0.6, 'beta': 0.4, 'beta_increment_per_sampling': 0.001, 'TD_err_

In [4]:
# data_dir = 'test_sets/synthetic_nrange_10_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_15_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_40_50_1000/'
# data_dir = 'valid_sets/synthetic_nrange_10_20_200/'
# data_dir = 'valid_sets/synthetic_nrange_15_20_200/'
data_dir = 'test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
data_dir_0 = 'test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
data_dir_1 = 'test_sets/tsp_min-n=40_max-n=50_num-graph=1000_type=random/'
data_dir_2 = 'test_sets/tsp_min-n=50_max-n=100_num-graph=1000_type=random/'
data_dir_3 = 'test_sets/tsp_min-n=100_max-n=200_num-graph=1000_type=random/'
test_dirs = [data_dir_0, data_dir_1, data_dir_2]

In [5]:
# load test data
graph_list, fnames = prepare_testset_S2VDQN(data_dir)
lengths, solutions, sol_times = run_test(dqn, graph_list)
approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, fnames, lengths)
print(mean_approx_ratio)
save_approx_ratios(data_dir, fnames, approx_ratios, model_file)
save_solutions(data_dir, fnames, solutions, model_file)
save_lengths(data_dir, fnames, lengths, model_file)


14it [00:00, 138.98it/s]Loading test graphs...
1000it [00:05, 182.11it/s]
 27%|██▋       | 270/1000 [00:17<00:45, 15.94it/s]

KeyboardInterrupt: 

In [5]:
mean_approx_ratios = []
for data_dir in test_dirs:
    # load test data
    graph_list, fnames = prepare_testset_S2VDQN(data_dir)
    lengths, solutions, sol_times = run_test(dqn, graph_list)
    approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, fnames, lengths)
    print("Mean approx ratio:", mean_approx_ratio)
    mean_approx_ratios.append(mean_approx_ratio)
    save_approx_ratios(data_dir, fnames, approx_ratios, model_file)
    save_solutions(data_dir, fnames, solutions, model_file)
    save_lengths(data_dir, fnames, lengths, model_file)

19it [00:00, 184.10it/s]Loading test graphs...
1000it [00:05, 191.61it/s]
100%|██████████| 1000/1000 [01:03<00:00, 15.73it/s]
Mean approx ratio: 1.0206159817090894


NameError: name 'best_model_file' is not defined

In [ ]:
print(mean_approx_ratios)

In [14]:
graph = graph_list[0]
length, solution, time_ = dqn.Evaluate(g=graph)

In [15]:
solution, length

([0, 6, 1, 11, 8, 9, 2, 7, 5, 10, 4, 3], 2.7711)

In [11]:
# check whether length calculation is correct
def calc_tour_length(graph, solution):
    tot_len = 0
    for i in range(np.array(solution).shape[0]):
        if i == np.array(solution).shape[0] - 1:
            tot_len += graph[solution[i]][solution[0]]['weight']
        else:
            tot_len += graph[solution[i]][solution[i + 1]]['weight']
    return tot_len

In [12]:
def add_node(graph, cur_sol, new_node):
    cur_dist = 10000000.0
    for i in range(0, np.array(cur_sol).shape[0]):
        if i + 1 == np.array(cur_sol).shape[0]:
            adj = cur_sol[0]
        else:
            adj = cur_sol[i + 1]
        if cur_sol[i] == adj:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight']
        else:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight'] - graph[cur_sol[i]][adj]['weight']
        if cost < cur_dist:
            cur_dist = cost
            pos = i + 1
    cur_sol.insert(pos, new_node)
    return  cur_dist

In [10]:
def calc_helper_tour(g):
    cur_sol = [0]
    rem_nodes = list(np.arange(g.number_of_nodes()))
    rem_nodes.remove(0)
    tour_length = 0
    while np.array(cur_sol).shape[0] < g.number_of_nodes():
        if np.array(rem_nodes).shape[0]-1 == 0:
            rand_idx = 0
        else:
            rand_idx = np.random.randint(0, np.array(rem_nodes).shape[0]-1)
        new_node = rem_nodes[rand_idx]
        tour_length += add_node(g, cur_sol, new_node)
        rem_nodes.remove(new_node)
    return tour_length, cur_sol


In [16]:
max_result = -np.inf
min_result = np.inf
for i in range(100):
    help_lengths = []
    for graph in graph_list:
        help_lengths.append(calc_helper_tour(graph)[0])

    true_lengths = None
    with open(data_dir+'lengths.txt', 'r') as f:
        lines = f.readlines()
        lines = [float(line.split(':')[-1].strip()) for line in lines]
    true_lengths = lines
    result = np.mean([length[0]/length[1] for length in zip(help_lengths, true_lengths)])
    if result < min_result:
        min_result = result
    if result > max_result:
        max_result = result

In [18]:
print(min_result)
print(max_result)

1.0291948212594315
1.0353626379176066


5.9577